# TensorFlow による RNN 実習<br><small>ハンズオン資料</small>

<p style="text-align:center;font-size:150%;line-height:150%">2016/09/10 機械学習 名古屋 第6回勉強会</p>

## はじめに

+ この資料は、TensorFlow を用いて、簡単な RNN の構築・学習を実施することを目的とするものです。  
+ この資料に掲載のコードは、TensorFlow 公式のチュートリアル [Recurrent Neural Networks](https://www.tensorflow.org/versions/master/tutorials/recurrent/) を元に構築しております。

※ 時間があったら、sherjilozair 氏の [char-rnn-tensorflow](https://github.com/sherjilozair/char-rnn-tensorflow) も取り込みたい。

## 目標

+ TensorFlow を用いた基本的な RNN が構築できる。
+ …（オプション、もしくは宿題）

## 環境等

以下の環境を前提とします。

+ Python（必須）（2.7.x / 3.x どちらでもOK）
+ TensorFlow（必須）（0.7 / 0.8 / 0.9 / 0.10[New!] どれでもOK）
+ IPython（任意）

※ TensorFlow 0.6（以前）は未対応。

## TensorFlow

+ [公式サイト](https://www.tensorflow.org/)
+ Google 製の「データフローグラフを用いた数値計算ライブラリ」（公式の説明を私訳）
  + DeepLearning 用の機能も豊富。
+ 最新 [v0.10](https://github.com/tensorflow/tensorflow/releases/tag/v0.10.0rc0) がリリース準備中(2016/09/03 現在、v0.10-rc0 公開中)

インストールの詳細省略。  
インストールが成功していれば、Python のインタラクティブシェル（もしくは ipython, Jupyter 等）で↓以下のようにすれば利用開始。

In [ ]:
import tensorflow as tf

※ 今回は TensorBoard は不使用。

## RNN

### おさらい

+ RNN：
    + 系列データ（文章、音声、映像等）のコンテキストを学習・分類するモデルの総称
+ コンテキスト（文脈）：
    + 系列内の要素の並び・依存関係
    + 要素の例：
        + 「文章」中の「文字」、「単語」、「形態素」等
        + 「音声」中の「（パワースペクトル密度係数の）ベクトル」、「音素」等
        + 「映像（動画）」中の「（ある時刻における）画像（静止画）」等

+ LSTM：
    + RNN の拡張モデル
    + メモリセルと3つのゲートからなるLSTMブロックという構造を持つ
    + （従来の RNN と比較して）「長期にわたる依存性」を学習可能

## PTB データセット

+ PTB データセット：
    + 単語の履歴を元に、文章中の次の単語を予測する「文章予測モデル」のための（サンプル）データセット。


※ RNN の最初のサンプル構築・実験に最適。

### データ準備

http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz をダウンロード・展開。  
（その中の `data/ptb.train.txt`, `data/ptb.valid.txt`, `data/ptb.test.txt` のみ使用）

In [ ]:
def _read_words(filename):
    with tf.gfile.GFile(filename, "r") as f:
        return f.read().replace("\n", "<eos>").split()


In [ ]:
def _build_vocab(filename):
    import collections

    data = _read_words(filename)

    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))

    return word_to_id


In [ ]:
def _file_to_word_ids(filename, word_to_id):
    data = _read_words(filename)
    return [word_to_id[word] for word in data]


In [ ]:
def ptb_raw_data(data_path=None):
    """Load PTB raw data from data directory "data_path"."""
    import os

    train_path = os.path.join(data_path, "ptb.train.txt")
    valid_path = os.path.join(data_path, "ptb.valid.txt")
    test_path = os.path.join(data_path, "ptb.test.txt")

    word_to_id = _build_vocab(train_path)
    train_data = _file_to_word_ids(train_path, word_to_id)
    valid_data = _file_to_word_ids(valid_path, word_to_id)
    test_data = _file_to_word_ids(test_path, word_to_id)
    vocabulary = len(word_to_id)
    return train_data, valid_data, test_data, vocabulary


In [ ]:
def ptb_iterator(raw_data, batch_size, num_steps):
    """Iterate on the raw PTB data."""
    import numpy as np
    
    raw_data = np.array(raw_data, dtype=np.int32)

    data_len = len(raw_data)
    batch_len = data_len // batch_size
    data = np.zeros([batch_size, batch_len], dtype=np.int32)
    for i in range(batch_size):
        data[i] = raw_data[batch_len * i:batch_len * (i + 1)]

    epoch_size = (batch_len - 1) // num_steps

    if epoch_size == 0:
        raise ValueError("epoch_size == 0, decrease batch_size or num_steps")

    for i in range(epoch_size):
        x = data[:, i*num_steps:(i+1)*num_steps]
        y = data[:, i*num_steps+1:(i+1)*num_steps+1]
        yield (x, y)


## モデル構築

### サンプルモデル定義データ

In [ ]:
class SampleTinyConfig(object):
    """Tiny config."""
    init_scale = 0.1
    learning_rate = 1.0
    max_grad_norm = 5
    num_layers = 2
    num_steps = 10
    hidden_size = 50
    max_epoch = 1
    max_max_epoch = 3
    keep_prob = 1.0
    lr_decay = 0.5
    batch_size = 20
    vocab_size = 10000


### モデル定義

In [ ]:
def ptbmodel_init_step1(config):
    """Prepare Placeholders."""
    _input_data = tf.placeholder(tf.int32, [config.batch_size, config.num_steps])
    _targets = tf.placeholder(tf.int32, [config.batch_size, config.num_steps])
    return (_input_data, _targets)


In [ ]:
def ptbmodel_init_step2(config, is_training):
    """Construct LSTM cell(s)."""
    batch_size = config.batch_size
    size = config.hidden_size
    vocab_size = config.vocab_size

    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(size, forget_bias=0.0)
    if is_training and config.keep_prob < 1:
        lstm_cell = tf.nn.rnn_cell.DropoutWrapper(
                lstm_cell, output_keep_prob=config.keep_prob)
    cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * config.num_layers)
    _initial_state = cell.zero_state(batch_size, tf.float32)

    return (cell, _initial_state)

※ 複数LSTMスタック（`tf.nn.rnn_cell.MultiRNNCell`）利用。  
※ ↑第1層の出力が第2層の入力になる。

In [ ]:
def ptbmodel_init_step3(config, _input_data, is_training):
    """Convert Input Data."""
    size = config.hidden_size
    vocab_size = config.vocab_size

    with tf.device("/cpu:0"):
        embedding = tf.get_variable("embedding", [vocab_size, size])
        inputs = tf.nn.embedding_lookup(embedding, _input_data)

    if is_training and config.keep_prob < 1:
        inputs = tf.nn.dropout(inputs, config.keep_prob)

    return inputs

※ 「単語のベクトル表現」利用（前回資料参照）

In [ ]:
def ptbmodel_init_step4(config, cell, inputs, _initial_state, _targets):
    """Calculate Cost and Final State."""
    batch_size = config.batch_size
    size = config.hidden_size
    vocab_size = config.vocab_size

    outputs = []
    state = _initial_state
    with tf.variable_scope("RNN"):
        for time_step in range(config.num_steps):
            if time_step > 0: tf.get_variable_scope().reuse_variables()
            (cell_output, state) = cell(inputs[:, time_step, :], state)
            outputs.append(cell_output)

    output = tf.reshape(tf.concat(1, outputs), [-1, size])
    softmax_w = tf.get_variable("softmax_w", [size, vocab_size])
    softmax_b = tf.get_variable("softmax_b", [vocab_size])
    logits = tf.matmul(output, softmax_w) + softmax_b
    loss = tf.nn.seq2seq.sequence_loss_by_example(
            [logits],
            [tf.reshape(_targets, [-1])],
            [tf.ones([batch_size * config.num_steps])])
    cost = tf.reduce_sum(loss) / batch_size
    return (cost, state)


In [ ]:
def ptbmodel_init_step5(config, cost):
    """Define Training Operation."""
    _lr = tf.Variable(0.0, trainable=False)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),
                                      config.max_grad_norm)
    optimizer = tf.train.GradientDescentOptimizer(_lr)
    _train_op = optimizer.apply_gradients(zip(grads, tvars))
    return (_lr, _train_op)


In [ ]:
class PTBModel(object):
    """The PTB model."""

    def __init__(self, is_training, config):
        self.batch_size = batch_size = config.batch_size
        self.num_steps = num_steps = config.num_steps
        size = config.hidden_size
        vocab_size = config.vocab_size

        _input_data, _targets = ptbmodel_init_step1(config)
        self.input_data = _input_data
        self.targets = _targets

        cell, _initial_state = ptbmodel_init_step2(config, is_training)
        self.initial_state = _initial_state

        inputs = ptbmodel_init_step3(config, _input_data, is_training)

        cost, _final_state = ptbmodel_init_step4(config, cell, inputs, _initial_state, _targets)
        self.cost = cost
        self.final_state = _final_state

        if not is_training:
            return

        _lr, _train_op = ptbmodel_init_step5(config, cost)
        self.lr = _lr
        self.train_op = _train_op

    def assign_lr(self, session, lr_value):
        session.run(tf.assign(self.lr, lr_value))


## 実行

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

import numpy as np
import tensorflow as tf


In [ ]:
def run_epoch(session, model, data, eval_op, verbose=False):
    """Runs the model on the given data."""
    epoch_size = ((len(data) // model.batch_size) - 1) // model.num_steps
    start_time = time.time()
    costs = 0.0
    iters = 0
    # state = model.initial_state.eval()
    state = session.run(model.initial_state)
    for step, (x, y) in enumerate(ptb_iterator(data, model.batch_size,
                                               model.num_steps)):
        cost, state, _ = session.run([model.cost, model.final_state, eval_op],
                                     {model.input_data: x,
                                        model.targets: y,
                                        model.initial_state: state})
        costs += cost
        iters += model.num_steps

        if verbose and step % (epoch_size // 10) == 10:
            print("%.3f perplexity: %.3f speed: %.0f wps" %
                  (step * 1.0 / epoch_size, np.exp(costs / iters),
                   iters * model.batch_size / (time.time() - start_time)))

    return np.exp(costs / iters)


In [ ]:
data_path = 'simple-examples/data/'
train_data, valid_data, test_data, _ = ptb_raw_data(data_path)

config = SampleTinyConfig()
eval_config = SampleTinyConfig()
eval_config.batch_size = 1
eval_config.num_steps = 1


In [ ]:
with tf.Graph().as_default(), tf.Session() as session:
    initializer = tf.random_uniform_initializer(-config.init_scale,
                                                config.init_scale)
    with tf.variable_scope("model", reuse=None, initializer=initializer):
        m = PTBModel(is_training=True, config=config)
    with tf.variable_scope("model", reuse=True, initializer=initializer):
        mvalid = PTBModel(is_training=False, config=config)
        mtest = PTBModel(is_training=False, config=eval_config)

    tf.initialize_all_variables().run()

    for i in range(config.max_max_epoch):
        lr_decay = config.lr_decay ** max(i - config.max_epoch, 0.0)
        m.assign_lr(session, config.learning_rate * lr_decay)

        print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
        train_perplexity = run_epoch(session, m, train_data, m.train_op,
                                                                 verbose=True)
        print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))
        valid_perplexity = run_epoch(session, mvalid, valid_data, tf.no_op())
        print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, valid_perplexity))

    test_perplexity = run_epoch(session, mtest, test_data, tf.no_op())
    print("Test Perplexity: %.3f" % test_perplexity)


※ 30分くらいかかります。  
※ Tensorflow v0.9以上 だと、WARNING が出力されますが、正常に動作します。  
※ 最終的に Test Perplexity の値が 155前後 で表示されるはずです。

## 参考

+ [Recurrent Neural Networks](https://www.tensorflow.org/tutorials/recurrent/index.html)（TensorFlow 公式チュートリアル）
    + [参考日本語訳その1](http://qiita.com/KojiOhki/items/149f96bd98973bd219ac)
    + [参考日本語訳その2 + コード解説](http://tensorflow.classcat.com/2016/03/13/tensorflow-cc-recurrent-neural-networks/)